In [ ]:
import torch
import os
import time
import json
## Load Config
with open('config/videos.json') as config_file:
    videos = json.load(config_file)
with open('config/name_to_url.json') as config_file:
    name_to_url = json.load(config_file)


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

torch.cuda.empty_cache()
device = "cuda:0" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", torch_dtype=torch.float16, attn_implementation="flash_attention_2").to(device)

# tokenizer = AutoTokenizer.from_pretrained("TabbyML/StarCoder-7B")
# model = AutoModelForCausalLM.from_pretrained("TabbyML/StarCoder-7B")

In [ ]:
# Variable Setup
shots = ["hashing", "react_svelte"]
torch.cuda.empty_cache()

# folder_no = len(os.listdir(f'data/outputs'))
folder_no = 1
os.makedirs(f'data/outputs/{folder_no}/text', exist_ok=True)
os.makedirs(f'data/outputs/{folder_no}/embeddings', exist_ok=True)
os.makedirs(f'data/outputs/{folder_no}/results', exist_ok=True)

# Write some metadata
with open(f'data/outputs/{folder_no}/metadata.txt', 'w') as f:
    f.write(f"Model: mistralai/Mistral-7B-Instruct-v0.1\n")
    f.write(f"Time: {time.time()}\n")
    f.write(f"Videos: {videos}\n")
    f.write(f"Shots: {shots}\n")

transcripts_dir = "data/transcripts/processed"
prompt_path = "data/prompts/prompt.txt"
oneshots_dir = "data/oneshots"
outputs_dir = f'data/outputs/{folder_no}'

In [ ]:
## All Inference
metadata = open(f'{outputs_dir}/metadata.txt', 'w')

## open all files in data/transcripts/processed. For each file, create a message using as many shots as given above. 
for file in os.listdir(transcripts_dir):
    target = file.split('.')[0]
    with open(f'{transcripts_dir}/{target}.txt', 'r', encoding='utf-8') as f_target_transcript:
        with open(prompt_path, 'r', encoding='utf-8') as f_prompt:
            prompt = f_prompt.read()

            ## Build messages for multi-shot
            messages = []
            for shot in shots:
                f_oneshot_transcript = open(f'{transcripts_dir}/{shot}.txt', 'r', encoding='utf-8')
                f_oneshot = open(f'{oneshots_dir}/{shot}.txt', 'r', encoding='utf-8')
                messages.append({"role": "user", "content": f'{prompt}{f_oneshot_transcript.read()[0:5000]}'})
                messages.append({"role": "assistant", "content": f_oneshot.read()})
                f_oneshot.close()
                f_oneshot_transcript.close()

            ## Append final message
            messages.append({"role": "user", "content": f'{prompt}{f_target_transcript.read()[0:5000]}'},)

    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)    
    print("Token length: ", len(inputs[0]))
    metadata.write(f"Target: {target}, Tokens: {len(inputs[0])}\n")

    ## An idea for later, do not include the oneshot prompt if we are only directly using transcript embedding
    # start_token = len(inputs[0]) 

    with torch.no_grad():
        start_time = time.time()
        text = model.generate(inputs, max_new_tokens=1500, do_sample=True)
        decoded = tokenizer.batch_decode(text)
        print(f'Decoding finished: {target} in {round(time.time() - start_time, 3)} seconds')   
        metadata.write(f"    Decoding took: {round(time.time() - start_time, 3)} seconds\n")
    ## obtain all tokens after the second "[/INST]" and remove the </s> token. Write this as our output.
    with open(f'{outputs_dir}/text/{target}.txt', 'w', encoding='utf-8') as f:
        f.write(decoded[0].split('[/INST]')[-1][1:-4])


    ## Free up memory
    del text
    del decoded
    torch.cuda.empty_cache()
    print(f'cuda memory allocated: {torch.cuda.memory_allocated()/1024**3:.2f} GB', f'cuda memory cached: {torch.cuda.memory_cached()/1024**3:.2f} GB')
    metadata.write(f'    cuda memory allocated: {torch.cuda.memory_allocated()/1024**3:.2f} GB, cuda memory cached: {torch.cuda.memory_cached()/1024**3:.2f} GB\n')

metadata.close()



In [ ]:
# Create embeddings from outputs
metadata = open(f'{outputs_dir}/metadata.txt', 'w')

for file in os.listdir(f'{outputs_dir}/text'):
    print(f'cuda memory allocated: {torch.cuda.memory_allocated()/1024**3:.2f} GB', f'cuda memory cached: {torch.cuda.memory_cached()/1024**3:.2f} GB')

    target = file.split('.')[0]
    start_time = time.time()
    with open(f'{outputs_dir}/text/{target}.txt', 'r', encoding='utf-8') as f:
        inputs = tokenizer(f.read(), return_tensors="pt").to(device)     
    with torch.no_grad():
        hidden = model(**inputs, output_hidden_states=True)
    print(f'Hidden states finished: {target} in {round(time.time() - start_time, 3)} seconds')
    metadata.write(f"Target: {target}, Hidden states took: {round(time.time() - start_time, 3)} seconds\n")

    ## Write hidden states
    ## Cut off all tokens before the first start token to remove oneshot and prompt.
    tensor_t = hidden.hidden_states[-1].transpose(1,2)
    # max_pool_t = torch.nn.functional.max_pool1d(tensor_t, tensor_t.shape[2])
    # avg_pool_t = torch.nn.functional.avg_pool1d(tensor_t, tensor_t.shape[2])
    max_pool = torch.nn.functional.max_pool1d(tensor_t, tensor_t.shape[2]).transpose(1, 2).squeeze()
    avg_pool = torch.nn.functional.avg_pool1d(tensor_t, tensor_t.shape[2]).transpose(1, 2).squeeze()
    print(max_pool.shape)
    torch.save(max_pool, f'{outputs_dir}/embeddings/max_{target}.pt')
    torch.save(avg_pool, f'{outputs_dir}/embeddings/avg_{target}.pt')

    print(f'Target: {target} finished. Wrote to file.')
    metadata.write(f'    Target: {target} tensor finished. Wrote to file.\n')

    del inputs
    del hidden
    del tensor_t
    del max_pool
    del avg_pool
    print(f'cuda memory allocated: {torch.cuda.memory_allocated()/1024**3:.2f} GB', f'cuda memory cached: {torch.cuda.memory_cached()/1024**3:.2f} GB')
    torch.cuda.empty_cache()





In [ ]:
### catan.txt
## Clean transcript, with new lines
## Char: 9388, Word: 1840, Tokens: 2483, Runtime: 3m 4.2s on Pytorch MPS, T/S = 13.5

## Clean transcript, no new lines
## Char: 9388, Word: 1840, Tokens: 2174, Runtime: 2m 43.9s on Pytorch MPS, T/S = 13.25

## Clean transcript, charging
## Char: 9388, Word: 1840, Tokens: 2174, Runtime: 2m 32.8s on Pytorch MPS

#### On A10s
# 8,000-token limit for Mistral-7B

### catan.txt
## Tokens: 2174, Runtime: 48.8s, T/S = 44.55

### mixtral8x7b.txt
## Tokens: 16981, Runtime: 13m 8.4s, T/S = 21.59

### mlops_llm_eval.txt
## Tokens: 10993, Runtime: 7m 5.1s, T/S = 25.87

### typescript_fireship.txt
## Tokens: 1042, Runtime: 27.6s, T/S = 37.75

### localized_deployment.txt | Tokens: 892,
## A10 // Runtime: 21.6s, T/S = 41.3
## M1  // Runtime:
